<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [34]</a>'.</span>

## Notebook set up

**Your task**: Apply at least two different feature engineering techniques to the `housing_df` dataframe to improve the dataset. At the end of the notebook, your engineered dataset and the original dataset will be used to train a linear regression model to predict `MedHouseVal`. Your goal is to achieve better model performance via feature engineering.

**Note**: If you have read ahead or you are familiar with the basics of training ML models, no there is no train-test split and yes, this means data leakage/genralizability is a concern. We will cover those topics in the next unit. For now, the goal is to keep things simple while still giving you an idea of how your feature engineering effects model performance.

Before applying transformations, explore the dataset to understand what techniques would be most beneficial.

### Import libraries

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
# Set random seed for reproducibility
np.random.seed(315)

### Load dataset

In [2]:
# Load California housing dataset
original_housing_df = pd.read_csv('https://gperdrizet.github.io/FSA_devops/assets/data/unit2/california_housing.csv')
housing_df = original_housing_df.copy()

## Task 1: Explore the dataset

Before deciding what feature engineering techniques to apply, explore the dataset to understand its characteristics.

**Things to investigate**:
- Display basic information about the dataset (`.info()`, `.describe()`)
- Check for missing values
- Examine feature distributions (histograms, box plots)
- Look at feature scales and ranges

Use this exploration to inform your feature engineering decisions in the following tasks.

In [3]:
housing_df.head()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [5]:
housing_df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [6]:
housing_df.isna().sum()

MedInc         0
HouseAge       0
AveRooms       0
AveBedrms      0
Population     0
AveOccup       0
Latitude       0
Longitude      0
MedHouseVal    0
dtype: int64

In [7]:
def logit(x):
    return np.log1p(x)

In [8]:
def plot_hist_and_log(df, count_cols, bins=50, include_log=False):
    """
    For each column in count_cols, plot:
      1) Regular histogram with mean, median, and mode lines (all red)
      2) log1p-transformed histogram (handling zeros/negatives with a shift),
         also with mean, median, and mode lines (all red) if include_log is True.
    Saves both plots as PNGs.
    """
    for c in count_cols:
        # Drop NaNs and ensure float
        x = np.asarray(df[c].dropna(), dtype=float)
        if x.size == 0:
            print(f"Column {c} has no non-NaN values, skipping.")
            continue

        # --- Statistics for original data ---
        mean_val = np.mean(x)
        median_val = np.median(x)
        mode_series = pd.Series(x).mode()
        mode_val = mode_series.iloc[0] if not mode_series.empty else None

        # --- Regular histogram ---
        fig, ax = plt.subplots()
        ax.hist(x, bins=bins)

        # Vertical lines for mean, median, mode (all red)
        ax.axvline(mean_val,   color="red", linestyle="--", label=f"Mean: {mean_val:.2f}")
        ax.axvline(median_val, color="red", linestyle="-.", label=f"Median: {median_val:.2f}")
        if mode_val is not None:
            ax.axvline(mode_val, color="red", linestyle=":", label=f"Mode: {mode_val:.2f}")

        ax.set_title(f"Histogram: {c}")
        ax.set_xlabel(c)
        ax.set_ylabel("Count")
        ax.legend()
        plt.show()

        if not include_log:
            continue

        # --- Log1p histogram (with shift so we can take log) ---
        x_log = logit(x)

        # Stats in log space
        mean_log = np.mean(x_log)
        median_log = np.median(x_log)
        mode_log_series = pd.Series(x_log).mode()
        mode_log = mode_log_series.iloc[0] if not mode_log_series.empty else None

        fig, ax = plt.subplots()
        ax.hist(x_log, bins=bins)

        ax.axvline(mean_log,   color="red", linestyle="--", label=f"Mean (log): {mean_log:.2f}")
        ax.axvline(median_log, color="red", linestyle="-.", label=f"Median (log): {median_log:.2f}")
        if mode_log is not None:
            ax.axvline(mode_log, color="red", linestyle=":", label=f"Mode (log): {mode_log:.2f}")

        ax.set_title(f"Histogram (log1p): {c}")
        ax.set_xlabel(f"log1p({c})")
        ax.set_ylabel("Count")
        ax.legend()
        plt.show()


In [9]:
coast_lat = 36.8
coast_lon = -122.0
housing_df["DistToCoast2"] = np.sqrt(
    (housing_df["Latitude"] - coast_lat)**2 +
    (housing_df["Longitude"] - coast_lon)**2
)

In [10]:
housing_df["LatLon"] = housing_df["Latitude"] * housing_df["Longitude"]
housing_df["LatDivLon"] = housing_df["Latitude"] / housing_df["Longitude"]

In [11]:

housing_df["PopDensity"] = housing_df["Population"] / (
    (housing_df["Latitude"] - housing_df["Latitude"].mean())**2 +
    (housing_df["Longitude"] - housing_df["Longitude"].mean())**2 + 1
)

In [12]:
numeric_cols = [
    "MedInc", "HouseAge", "AveRooms", "AveBedrms",
    "Population", "AveOccup", "Latitude", "Longitude",
    "MedHouseVal", "BedroomsPerOccupant", "RoomsPerOccupant",
    "BedrmsPerRoom", "RoomsPerPerson", "PopPerHousehold", "DistToCoast", 
    "DistToCoast2", "LatLon","LatDivLon", "PopDensity"
]

num_cols = housing_df.select_dtypes(include=["number"]).columns


In [13]:
plot_hist_and_log(housing_df, num_cols,include_log =False)

In [14]:
 n_bins = 10

# Equal-width bins for Latitude
lat_edges = np.linspace(housing_df["Latitude"].min(),
                        housing_df["Latitude"].max(),
                        n_bins + 1)

housing_df["Lat_bin_idx"] = pd.cut(
    housing_df["Latitude"],
    bins=lat_edges,
    labels=False,          # <- gives 0,1,2,... instead of (37.81, 38.48]
    include_lowest=True
)

# Normalize to [0,1]
housing_df["Lat_bin_norm"] = housing_df["Lat_bin_idx"] / (n_bins - 1)

In [15]:
housing_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MedInc        20640 non-null  float64
 1   HouseAge      20640 non-null  float64
 2   AveRooms      20640 non-null  float64
 3   AveBedrms     20640 non-null  float64
 4   Population    20640 non-null  float64
 5   AveOccup      20640 non-null  float64
 6   Latitude      20640 non-null  float64
 7   Longitude     20640 non-null  float64
 8   MedHouseVal   20640 non-null  float64
 9   DistToCoast2  20640 non-null  float64
 10  LatLon        20640 non-null  float64
 11  LatDivLon     20640 non-null  float64
 12  PopDensity    20640 non-null  float64
 13  Lat_bin_idx   20640 non-null  int64  
 14  Lat_bin_norm  20640 non-null  float64
dtypes: float64(14), int64(1)
memory usage: 2.4 MB


## Task 2: Apply your first feature engineering technique

Based on your exploration, apply your first feature engineering technique.

**Example approaches**:
- Transform skewed features using log, sqrt, power, or quantile transformations
- Create bins/categories from continuous variables
- Create interaction features (e.g., rooms per household = total rooms / households)

In [16]:
# YOUR CODE HERE
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MedInc        20640 non-null  float64
 1   HouseAge      20640 non-null  float64
 2   AveRooms      20640 non-null  float64
 3   AveBedrms     20640 non-null  float64
 4   Population    20640 non-null  float64
 5   AveOccup      20640 non-null  float64
 6   Latitude      20640 non-null  float64
 7   Longitude     20640 non-null  float64
 8   MedHouseVal   20640 non-null  float64
 9   DistToCoast2  20640 non-null  float64
 10  LatLon        20640 non-null  float64
 11  LatDivLon     20640 non-null  float64
 12  PopDensity    20640 non-null  float64
 13  Lat_bin_idx   20640 non-null  int64  
 14  Lat_bin_norm  20640 non-null  float64
dtypes: float64(14), int64(1)
memory usage: 2.4 MB


In [17]:
skewed_cols = ["MedInc", "AveRooms", "AveBedrms", "Population", "AveOccup"]
for col in skewed_cols:
    housing_df[col] = logit(housing_df[col])

In [18]:
housing_df["RoomsPerBedroom"] = housing_df["AveRooms"] / housing_df["AveBedrms"]
housing_df["BedroomsPerOccupant"] = housing_df["AveBedrms"] / housing_df["AveOccup"]
housing_df["RoomsPerOccupant"] = housing_df["AveRooms"] / housing_df["AveOccup"]

In [19]:
housing_df["BedrmsPerRoom"] = housing_df["AveBedrms"] / housing_df["AveRooms"]
housing_df["RoomsPerPerson"] = housing_df["AveRooms"] / housing_df["AveOccup"]
housing_df["PopPerHousehold"] = housing_df["Population"] / housing_df["AveOccup"]

In [20]:
lat_mean = housing_df["Latitude"].mean()
lon_mean = housing_df["Longitude"].mean()

# Distance to the "center" of the data (in degrees, not true km, but good enough as a feature)
housing_df["DistFromMean"] = np.sqrt(
    (housing_df["Latitude"] - lat_mean)**2 +
    (housing_df["Longitude"] - lon_mean)**2
)



In [21]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MedInc               20640 non-null  float64
 1   HouseAge             20640 non-null  float64
 2   AveRooms             20640 non-null  float64
 3   AveBedrms            20640 non-null  float64
 4   Population           20640 non-null  float64
 5   AveOccup             20640 non-null  float64
 6   Latitude             20640 non-null  float64
 7   Longitude            20640 non-null  float64
 8   MedHouseVal          20640 non-null  float64
 9   DistToCoast2         20640 non-null  float64
 10  LatLon               20640 non-null  float64
 11  LatDivLon            20640 non-null  float64
 12  PopDensity           20640 non-null  float64
 13  Lat_bin_idx          20640 non-null  int64  
 14  Lat_bin_norm         20640 non-null  float64
 15  RoomsPerBedroom      20640 non-null 

In [22]:
lat_cols =  ["LatRegion_Lat_Q1", "LatRegion_Lat_Q2", "LatRegion_Lat_Q3", "LatRegion_Lat_Q4"]
cost_col =  ["DistToCoast", "DistToCoast2"]
lat_bin_col =  ["Lat_bin_norm", "Lat_bin_idx"]
housing_df = housing_df.drop(lat_bin_col, axis=1)

In [23]:
# Simple interaction term
housing_df["LatRegion"] = pd.qcut(
    housing_df["Latitude"],
    q=4,
    labels=["Lat_Q1", "Lat_Q2", "Lat_Q3", "Lat_Q4"]
)


In [24]:
housing_df["DistToCoast"] = abs(housing_df["Longitude"] + 124)

In [25]:
housing_df["DistToCoast"]

0        1.77
1        1.78
2        1.76
3        1.75
4        1.75
         ... 
20635    2.91
20636    2.79
20637    2.78
20638    2.68
20639    2.76
Name: DistToCoast, Length: 20640, dtype: float64

In [26]:
#housing_df["Lat_Lon_Interaction"] = housing_df["Latitude"] * housing_df["Longitude"]

In [27]:
from sklearn.cluster import KMeans

# Choose how many regions you want (try 5, 8, 10, etc.)
N_CLUSTERS = 8

coords = housing_df[["Latitude", "Longitude"]]

kmeans = KMeans(
    n_clusters=N_CLUSTERS,
    random_state=42,
    n_init="auto"   # or an int like 10 if you get a warning
)

housing_df["LocationCluster"] = kmeans.fit_predict(coords)

housing_df["LocationCluster"].value_counts()

LocationCluster
0    8673
7    4783
5    2132
3    1518
1    1037
2    1016
4     964
6     517
Name: count, dtype: int64

## Task 3: Apply your second feature engineering technique

**Example approaches**:
- Scale features to similar ranges
- Encode any categorical variables you created
- Create aggregate statistics by groups

In [28]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
onehot_array = encoder.fit_transform(housing_df[['LatRegion']])

# Create dataframe with proper column names
df_onehot = pd.DataFrame(
    onehot_array,
    columns=encoder.get_feature_names_out(['LatRegion']),
    index=housing_df.index
)

df_onehot

housing_df = pd.concat([housing_df.drop("LatRegion", axis=1), df_onehot], axis=1)


In [29]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MedInc               20640 non-null  float64
 1   HouseAge             20640 non-null  float64
 2   AveRooms             20640 non-null  float64
 3   AveBedrms            20640 non-null  float64
 4   Population           20640 non-null  float64
 5   AveOccup             20640 non-null  float64
 6   Latitude             20640 non-null  float64
 7   Longitude            20640 non-null  float64
 8   MedHouseVal          20640 non-null  float64
 9   DistToCoast2         20640 non-null  float64
 10  LatLon               20640 non-null  float64
 11  LatDivLon            20640 non-null  float64
 12  PopDensity           20640 non-null  float64
 13  RoomsPerBedroom      20640 non-null  float64
 14  BedroomsPerOccupant  20640 non-null  float64
 15  RoomsPerOccupant     20640 non-null 

## (Optional) Additional feature engineering

Add more techniques if you'd like to experiment further.

In [30]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson')
housing_df["HouseAge_pt"] = pt.fit_transform(housing_df[["HouseAge"]])
housing_df["AveRooms_pt"] = pt.fit_transform(housing_df[["AveRooms"]])
housing_df["AveOccup_pt"] = pt.fit_transform(housing_df[["AveOccup"]])

In [31]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(output_distribution="uniform")
housing_df["Lat_uniform"] = qt.fit_transform(housing_df[["Latitude"]])
housing_df["Lon_uniform"] = qt.fit_transform(housing_df[["Longitude"]])
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
housing_df["Lat_z"] = sc.fit_transform(housing_df[["Latitude"]])
housing_df["Lon_z"] = sc.fit_transform(housing_df[["Longitude"]])
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   MedInc               20640 non-null  float64
 1   HouseAge             20640 non-null  float64
 2   AveRooms             20640 non-null  float64
 3   AveBedrms            20640 non-null  float64
 4   Population           20640 non-null  float64
 5   AveOccup             20640 non-null  float64
 6   Latitude             20640 non-null  float64
 7   Longitude            20640 non-null  float64
 8   MedHouseVal          20640 non-null  float64
 9   DistToCoast2         20640 non-null  float64
 10  LatLon               20640 non-null  float64
 11  LatDivLon            20640 non-null  float64
 12  PopDensity           20640 non-null  float64
 13  RoomsPerBedroom      20640 non-null  float64
 14  BedroomsPerOccupant  20640 non-null  float64
 15  RoomsPerOccupant     20640 non-null 

## Model evaluation

Now we'll compare model performance on the original dataset versus your engineered dataset.

### Evaluate datasets

In [32]:
# Create output directory if it doesn't exist
output_directory = 'data/outputs'
Path(output_directory).mkdir(parents=True, exist_ok=True)

# Save a copy of the engineered dataframe
housing_df.to_csv('data/outputs/housing_df.csv', index=False)

In [33]:
# Create linear regression model
model = LinearRegression()

# Evaluate on original dataset
scores_original = cross_val_score(
    model,
    original_housing_df.drop('MedHouseVal', axis=1),
    original_housing_df['MedHouseVal'],
    cv=10,
    scoring='r2'
)

# Evaluate on engineered dataset
scores_engineered = cross_val_score(
    model,
    housing_df.drop('MedHouseVal', axis=1),
    housing_df['MedHouseVal'],
    cv=10,
    scoring='r2'
)

engineered_mean = scores_engineered.mean()
original_mean = scores_original.mean()
mean_improvement = (engineered_mean - original_mean) / original_mean

print(f'\nMean improvement: {mean_improvement:.2f}%')


Mean improvement: 0.13%


### Visualize model performance comparison

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [34]:
original_model = LinearRegression()
original_model.fit(original_housing_df.drop('MedHouseVal', axis=1), original_housing_df['MedHouseVal'])
original_predictions = original_model.predict(original_housing_df.drop('MedHouseVal', axis=1))

model = LinearRegression()
model.fit(housing_df.drop('MedHouseVal', axis=1), housing_df['MedHouseVal'])
predictions = model.predict(housing_df.drop('MedHouseVal', axis=1))

# Create boxplot comparing performance
data_to_plot = [scores_original, scores_engineered]
labels = ['Original', 'Engineered']

fig, axs = plt.subplots(1, 2, figsize=(9,4.5))

fig.suptitle(f'Model performance comparison\nmean improvement: {mean_improvement:.2f}%')

axs[0].set_title('Cross validation R² scores')
axs[0].boxplot(data_to_plot, labels=labels)
axs[0].set_xlabel('Dataset')
axs[0].set_ylabel('R² score')

axs[1].set_title('Predictions vs true values')
axs[1].plot(
    original_housing_df['MedHouseVal'], original_predictions,
    'o', markersize=1, label='Original', alpha=0.25
)

axs[1].plot(
    housing_df['MedHouseVal'], predictions,
    'o', markersize=1, label='Engineered', alpha=0.25
)

axs[1].set_xlabel('True Values')
axs[1].set_ylabel('Predictions')

leg = axs[1].legend(loc='upper left', markerscale=8, framealpha=1)

for lh in leg.legendHandles: 
    lh.set_alpha(1)

plt.tight_layout()
plt.show()

/tmp/ipykernel_10870/2730283522.py:18: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  axs[0].boxplot(data_to_plot, labels=labels)


AttributeError: 'Legend' object has no attribute 'legendHandles'

## 3. Reflection

**Questions to consider**:

1. Which feature engineering techniques had the biggest impact on model performance?
2. Did adding more features always improve performance, or did some hurt it?
3. How might you further improve the engineered dataset?
4. What trade-offs did you consider (e.g., interpretability vs performance, complexity vs gains)?

**Your reflection**:

*Write your thoughts here...*